#  Практика по работе с Энкодерами

В этом задании мы потренируемся использовать различные энкодеры под наши задачи. Мы поработаем с классической бинарной классификацией отзывов на фильмы, попробуем усложнить и выйти на многоклассовую задачу, а после - подключим определение семантической близости фильмов по их описанию.

### Бинарная классификация

In [ ]:
!pip install --upgrade datasets transformers

In [ ]:
import numpy as np
import torch
import pandas as pd
from datasets import load_dataset, Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
    set_seed,
)

SEED = 42
set_seed(SEED)

Перед вами практическое задание по бинарной классификации текстов с использованием предобученной модели BERT. Мы будем решать задачу определения тональности пользовательских отзывов на фильмы — **положительный** отзыв или **отрицательный**.

В качестве источника данных используем открытый датасет [IMDb](https://huggingface.co/datasets/stanfordnlp/imdb) от [Stanford NLP](https://nlp.stanford.edu/), содержащий 50 000 англоязычных рецензий, размеченных вручную. Цель — построить и обучить модель, способную автоматически классифицировать тексты по их эмоциональной окраске. Вы пройдёте все основные этапы пайплайна: от очистки текста и токенизации до обучения модели и оценки качества.

Перед тем, как мы двинемся дальше, проверьте включен или выключен GPU в colab. Работа с BERT потребует ресурсов, но предлагаем сначала заполнить весь необходимый код, а потом уже подключить GPU перед самым обучением модели. Советуем использовать доступную в colab Tesla T4.

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"GPU на месте: {torch.cuda.get_device_name(0)}")

GPU на месте: Tesla T4


In [ ]:
# датасет сразу подгружен в модуль datasets от HF

dataset = load_dataset("imdb")

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

Мы видим, что в датасете уже подготовленное разделение данных на тренировочную и тестовую выборку, а также "unsupervised" набор - неразмеченная часть данных — то есть отзывы, у которых нет корректной метки (label не несёт обучающей информации).

* В части `train` и `test` у каждого отзыва есть метка: `label = 0` (негативный) или `label = 1` (позитивный).
* В части `unsupervised` колонка `label` есть, но её значение не используется — оно либо пустое, либо фиктивное (`-1`), так как эта часть предназначена для:

  * обучения без учителя (например, pretraining),
  * самостоятельного дообучения языковой модели,
  * полу- или слаборазмеченного обучения.

In [ ]:
dataset["unsupervised"][0] # лейбл обозначен как "-1"

{'text': 'This is just a precious little diamond. The play, the script are excellent. I cant compare this movie with anything else, maybe except the movie "Leon" wonderfully played by Jean Reno and Natalie Portman. But... What can I say about this one? This is the best movie Anne Parillaud has ever played in (See please "Frankie Starlight", she\'s speaking English there) to see what I mean. The story of young punk girl Nikita, taken into the depraved world of the secret government forces has been exceptionally over used by Americans. Never mind the "Point of no return" and especially the "La femme Nikita" TV series. They cannot compare the original believe me! Trash these videos. Buy this one, do not rent it, BUY it. BTW beware of the subtitles of the LA company which "translate" the US release. What a disgrace! If you cant understand French, get a dubbed version. But you\'ll regret later :)',
 'label': -1}

In [ ]:
dataset["unsupervised"][0]['text']
dataset["unsupervised"][0]['label']

-1

В нашей задаче мы не будем использовать `unsupervised` часть — только размеченные данные из `train` и `test`. Точнее, только из `train` - 50.000 объектов на обучение это неплохо, но для учебной задачи долговато. С целью сокращения времени - сократим и датасет.

**Задание 1.** Давайте оставим только `train`, а затем перемешаем его с помщью `shuffle`, а после разобьем на обучающую и валидационную выборки в соотношении 80:20.

> Попробуем удержать баланс классов? Установите в гиперпараметре пропорциональное распределение классов. Если не помните, как это выполнить, загляните в описание метода [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html).

In [ ]:
from sklearn.model_selection import train_test_split

df = dataset["train"].to_pandas()

train_df, val_df = train_test_split(
    df,
    test_size=0.2,           # 20% в валидацию
    shuffle=True,            # Перемешивание данных
    stratify=df['label'],         # Сохраняем баланс классов
    random_state=42          # Для воспроизводимости
)

print(f"Размер обучающей выборки: {len(train_df)}")
print(f"Размер валидационной выборки: {len(val_df)}")

Размер обучающей выборки: 20000
Размер валидационной выборки: 5000


**Вопрос 1.** Сколько отзывов положительного класса `(label = 1)` содержится в `val_df` после разбиения?

**Задание 2.** **А стоит ли чистить отзывы?**

Перед тем как обучать модель на текстовых данных, важно понять, в каком виде приходят тексты, и нужна ли им предобработка.

В нашем случае отзывы взяты напрямую из `IMDb` — они написаны живыми пользователями, без фильтрации и нормализации. Чтобы принять решение об очистке, посмотрите на первые несколько примеров из датасета.

Обратите внимание на:

* наличие заглавных букв,
* пунктуацию (много ли её? нарушает ли она понимание?),
* цифры и HTML-теги (встречаются ли? нужны ли они для тональности?),
* стоп-слова (могут ли они мешать?).

**Вопрос 2** - на размышление: *Какую базовую предобработку вы бы применили к этим отзывам перед подачей в токенизатор BERT?*

> *Важно помнить*: модели `BERT` обучаются на сырых текстах, но легкая очистка всё же может помочь — особенно, если мы визуализируем текст, считаем TF-IDF или хотим сделать простой анализ.

В следующем шаге вы напишете функцию очистки текста. Если вы считаете, что некоторые из этих элементов обработки не нужны, всё равно, пожалуйста, выполните их. В бонусном задании вы сможете вернуться к этому вопросу и поэкспериментировать с разными подходами к обработке.

**Задание 3.** Теперь почистим наши тексты. Напишите функцию `clean_text`, которая будет очищать текст от лишнего. Вы можете опираться на код с семинара или написать свой вариант. Давайте: 1) приведем к нижнему регистру 2) уберем числа 3) удалим переносы строк 4) уберем стоп-слова.

Также удалите из таблицы те строки, где текст отзыва оказался пустым после очистки.

Вам здесь пригодятся модули `re`, `string`, `nltk` (`nltk.download('stopwords')`), `stopwords` из `nltk.corpus`. Не забудьте применить обработку и к трейну, и к тесту.

In [ ]:
import pandas as pd
import numpy as np
import re # работа с регулярными выражениями (очистка текста)
import string

# Импорт библиотеки NLTK для работы с текстом
import nltk
from nltk.corpus import stopwords

from datasets import Dataset

from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

from transformers import Trainer, TrainingArguments
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

In [ ]:
def clean_text(text):
    text = text.lower()  # Приведение к нижнему регистру
    text = re.sub(f'[{string.punctuation}]', '', text)  # Удаление пунктуации
    text = re.sub('\n', ' ', text)  # Удаление переносов строк
    text = re.sub('\d+', '', text)  # Удаление цифр
    text = ' '.join([word for word in text.split() if word not in stop_words])  # Удаление стоп-слов
    return text

# # # Удаление строк с пустыми отзывами
# X.dropna(subset=['text'], inplace=True)

# # Стоп-слова
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

train_df['cleaned_text'] = train_df['text'].apply(clean_text)
val_df['cleaned_text'] = val_df['text'].apply(clean_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


**Вопрос 3.** Чему равна средняя длина очищенных отзывов (в символах) в `train_df`?

**Задание 4.** Теперь реализуем функцию оценки качества. При обучении модели через `Trainer` из библиотеки 🤗 можно передать свою функцию подсчёта метрик. Это позволяет отслеживать не только `loss`, но и, например, F1-score, accuracy или другие метрики качества.


Мы хотим реализовать функцию `compute_metrics`, которая будет передаваться в `Trainer`. Эта функция получает на вход кортеж `(logits, labels)` — предсказания модели и реальные метки, и должна возвращать **accuracy** (долю правильных ответов).

**Вопрос 4.** Почему, кстати, мы выбрали здесь именно **accuracy** метрику?



In [ ]:
# здесь импуты, если потребуются

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {'accuracy': accuracy_score(labels, predictions)}

Запустите следующий код. Если ваша функция написана верно, то код отработает без ошибок.

In [ ]:
test_logits = np.array([[0.1, 0.9], [0.8, 0.2], [0.4, 0.6]])
test_labels = np.array([1, 0, 1])

eval_pred = (test_logits, test_labels)

metrics = compute_metrics(eval_pred)
print("Test metrics:", metrics)

assert "accuracy" in metrics, "Функция должна возвращать словарь с ключом 'accuracy'"
assert isinstance(metrics["accuracy"], float), "Значение accuracy должно быть числом"
assert abs(metrics["accuracy"] - 1.0) < 1e-6, "Ожидаемое значение accuracy: 1.0 на идеально предсказанном примере"

Test metrics: {'accuracy': 1.0}


**Задание 5.** Реализуем функцию токенизации текста. Перед тем как подавать тексты в модель `BERT`, нужно преобразовать их в числовой формат — токены. Это делает токенизатор модели.

Токенизатор превращает каждый отзыв в набор чисел, соответствующих подсловным единицам, которые `BERT` использует как вход. Но важно задать параметры, чтобы все примеры имели одинаковую длину и не обрезались неконтролируемо.


Реализуйте функцию `tokenize_function`, которая применяет токенизатор к колонке с очищенными отзывами. Мы будем использовать *построчную токенизацию с паддингом и усечением*.


In [ ]:
def tokenize_function(examples):
    return tokenizer(examples['cleaned_text'],
                     padding='max_length',
                     truncation=True,
                     max_length=256)

**Вопрос 5.** Какой набор параметров токенизатора будет наиболее подходящим для обучения модели классификации отзывов?

Теперь давайте немного порассуждаем насчет гиперпараметра `max_length`: какое количество токенов мы будет отправлять в нашу модель? Итак, что мы уже знаем:
1. максимум для `BERT` это `max_length = 512`.
2. В английском языке при использовании токенизации WordPiece (у `bert-base-uncased`, например) из 100 символов в среднем получается от 20 до 40 токенов, в зависимости от слов, пунктуации и наличия редких подслов.



In [ ]:
# def tokenize_function(examples):
#     return tokenizer(examples['cleaned_text'],
#                      padding='max_length',
#                      truncation=True,
#                      max_length=256)

train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
text = train_dataset["cleaned_text"][0]
tokens = tokenizer.tokenize(text)

print(len(text), "символов")
print(len(tokens), "токенов")

# 1127 символов
# 223 токенов

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

NameError: name 'tokenizer' is not defined

Если средняя длина отзывов после очистки — около 900 символов, то разумный выбор:

* `max_length = 256` — безопасный и быстрый вариант
* `max_length = 384` — если хочется сохранить больше контекста
* `max_length = 512` — максимум для **BERT**, но:

  * увеличивает время и потребление памяти
  * может быть избыточен (для коротких отзывов)



**Задание 6.** Подобрались мы и к обучению. Теперь, когда вы освоили все этапы предобработки, токенизации и настройки модели, давайте применим эти знания на практике и сравним три разных версии BERT на задаче классификации отзывов.



Это будет самая сложная часть - но и самая содержательная. Цель задания:

1. Написать **полный пайплайн обучения модели**, включая токенизацию (подтягиваем нашу функцию, написанную выше), обучение, валидацию.
2. Запустить пайнлайн с тремя предобученными моделями:
   * `"bert-base-uncased"`
   * `"roberta-base"`
   * `"distilbert-base-uncased"`
3. Обучить каждую модель на 3 эпохах и сравнить метрику качества (`accuracy`).

PS Вы можете написать логику для каждой модели отдельно, но будет более изящно, если вы объедините всё в единый цикл - без лишнего дублирования кода.

> Почитайте, что такое [DataCollatorWithPadding](https://huggingface.co/docs/transformers/main_classes/data_collator#transformers.DataCollatorWithPadding:~:text=value%20at%20initialization.-,DataCollatorWithPadding,-class%20transformers.) и попробуйте интегрировать в своё решение.

In [ ]:
# Аргументы обучения - пусть будут едины, чтобы мы согли синхронизировать результаты
args = TrainingArguments(
    output_dir="checkpoints",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_strategy="epoch",
    warmup_ratio=0.1,
    weight_decay=0.01,
    fp16=True,
    report_to="none" # отключаем wandb и tensorboard - вы можете подключить, если это нужно
)

In [ ]:
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)

train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

train_dataset = train_dataset.remove_columns([ 'text', 'cleaned_text', '__index_level_0__'])
val_dataset = val_dataset.remove_columns([ 'text', 'cleaned_text', '__index_level_0__'])

train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask',  'label'])

# # Меняем название, поскольку модель ожидает название "labels"
train_dataset = train_dataset.rename_column('label', 'labels')
val_dataset = val_dataset.rename_column('label', 'labels')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [ ]:
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

training_args = TrainingArguments(
    output_dir="checkpoints",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_strategy="epoch",
    warmup_ratio=0.1,
    weight_decay=0.01,
    fp16=True,
    report_to="none" # отключаем wandb и tensorboard - вы можете подключить, если это нужно
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()
trainer.evaluate()

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.359900,0.382966,0.883800
2,0.247000,0.371098,0.904600
3,0.105200,0.475585,0.907200


{'eval_loss': 0.4755849540233612,
 'eval_accuracy': 0.9072,
 'eval_runtime': 18.4178,
 'eval_samples_per_second': 271.476,
 'eval_steps_per_second': 33.935,
 'epoch': 3.0}

# RoBERTa (roberta-base)

In [ ]:
model_name = "roberta-base"
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = RobertaForSequenceClassification.from_pretrained(model_name, num_labels=2)

train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

train_dataset = train_dataset.remove_columns([ 'text', 'cleaned_text', '__index_level_0__'])
val_dataset = val_dataset.remove_columns([ 'text', 'cleaned_text', '__index_level_0__'])

train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask',  'label'])

# # Меняем название, поскольку модель ожидает название "labels"
train_dataset = train_dataset.rename_column('label', 'labels')
val_dataset = val_dataset.rename_column('label', 'labels')

training_args = TrainingArguments(
    output_dir="checkpoints",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_strategy="epoch",
    warmup_ratio=0.1,
    weight_decay=0.01,
    fp16=True,
    report_to="none" # отключаем wandb и tensorboard - вы можете подключить, если это нужно
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()
trainer.evaluate()

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

...

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Accuracy
1,0.443900,0.380204,0.862200
2,0.309500,0.403328,0.885000
3,0.226000,0.353187,0.905200


{'eval_loss': 0.3531869351863861,
 'eval_accuracy': 0.9052,
 'eval_runtime': 18.0795,
 'eval_samples_per_second': 276.556,
 'eval_steps_per_second': 34.569,
 'epoch': 3.0}

### DistilBERT (distilbert-base-uncased)

In [ ]:
model_name = "distilbert-base-uncased"
tokenizer = DistilBertTokenizer.from_pretrained(model_name)
model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=2)

train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

train_dataset = train_dataset.remove_columns([ 'text', 'cleaned_text', '__index_level_0__'])
val_dataset = val_dataset.remove_columns([ 'text', 'cleaned_text', '__index_level_0__'])

train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask',  'label'])

# # Меняем название, поскольку модель ожидает название "labels"
train_dataset = train_dataset.rename_column('label', 'labels')
val_dataset = val_dataset.rename_column('label', 'labels')

training_args = TrainingArguments(
    output_dir="checkpoints",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_strategy="epoch",
    warmup_ratio=0.1,
    weight_decay=0.01,
    fp16=True,
    report_to="none" # отключаем wandb и tensorboard - вы можете подключить, если это нужно
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()
trainer.evaluate()

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Accuracy
1,0.325700,0.290508,0.892000
2,0.215700,0.353701,0.903800
3,0.091900,0.471402,0.903200


{'eval_loss': 0.47140172123908997,
 'eval_accuracy': 0.9032,
 'eval_runtime': 9.6525,
 'eval_samples_per_second': 518.002,
 'eval_steps_per_second': 64.75,
 'epoch': 3.0}

**Бонусное задание.** Выберете модель: можно взять из числа предложенных или любую другую - и попробуйте максимально увеличить метрики на валидации. Можно экспериментировать с методами обработки текстов, с архитектурами модели, подбором гиперпараметров и тд. Можно и учеличить объем данных - если вам позволит время и ресурсы.

Напишите о своих успехах нам на степике в комментариях под бонусным вопросом. И студентам, и нам будет очень интересно прочесть ваши эксперименты и результаты.

In [ ]:
# здесь ваш код
# ヽ(♡‿♡)ノ